<a href="https://colab.research.google.com/github/PANDASANG1231/deeplearn_note/blob/master/026_GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deeplearning_note')
from tool import *

Mounted at /content/drive


In [2]:
import torch
import torchvision

In [7]:
! nvidia-smi

Fri Nov 12 17:05:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
class Inception(torch.nn.Module):

    def __init__(self, in_channels, c1, c2, c3, c4):

        super().__init__()
        self.p1_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=c1, kernel_size=1, padding=0, stride=1)
        self.p2_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=c2[0], kernel_size=1, padding=0, stride=1)
        self.p2_2 = torch.nn.Conv2d(in_channels=c2[0], out_channels=c2[1], kernel_size=3, padding=1, stride=1)
        self.p3_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=c3[0], kernel_size=1, padding=0, stride=1)
        self.p3_2 = torch.nn.Conv2d(in_channels=c3[0], out_channels=c3[1], kernel_size=5, padding=2, stride=1)
        self.p4_1 = torch.nn.MaxPool2d(kernel_size=3, padding=1, stride=1)
        self.p4_2 = torch.nn.Conv2d(in_channels=in_channels, out_channels=c4, kernel_size=1, padding=0, stride=1)

    def forward(self, X):

        x1 = torch.nn.functional.relu(self.p1_1(X))
        x2 = torch.nn.functional.relu(self.p2_2(self.p2_1(X)))
        x3 = torch.nn.functional.relu(self.p3_2(self.p3_1(X)))
        x4 = torch.nn.functional.relu(self.p4_2(self.p4_1(X)))

        return torch.cat((x1, x2, x3, x4), 1)
        ## batch_size is dim-0, channels size is dim-1


from torch import nn

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

model = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))
    


In [12]:
transforms = torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Resize(size=96)
             ])

train_dataset = torchvision.datasets.FashionMNIST(root='./', train=True, download=True, transform=transforms)
test_dataset = torchvision.datasets.FashionMNIST(root='./', train=False, download=True, transform=transforms)

In [13]:
batch_size = 128
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False)

device = torch.device('cuda')
loss = torch.nn.CrossEntropyLoss()

print("This is a simple GoogLenet...")
x = torch.randn(size=(1, 1, 224, 224))
for layer in model:
  x = layer(x)
  print(layer.__class__.__name__, x.shape) 



This is a simple GoogLenet...
Sequential torch.Size([1, 64, 56, 56])
Sequential torch.Size([1, 192, 28, 28])
Sequential torch.Size([1, 480, 14, 14])
Sequential torch.Size([1, 832, 7, 7])
Sequential torch.Size([1, 1024])
Linear torch.Size([1, 10])


In [15]:
train_p2(epoch_num=10, 
         model=model,  
         loss=loss, 
         lr=0.05, 
         train_data_iter=train_dataloader,
         test_data_iter=test_dataloader,
         device=device,
         optim_type="SGD")

loss 1.458, train acc 0.470, test acc 0.711
loss 0.511, train acc 0.807, test acc 0.790
loss 0.389, train acc 0.853, test acc 0.852
loss 0.335, train acc 0.874, test acc 0.877
loss 0.306, train acc 0.884, test acc 0.878
loss 0.282, train acc 0.893, test acc 0.880
loss 0.263, train acc 0.900, test acc 0.877
loss 0.247, train acc 0.907, test acc 0.894
loss 0.235, train acc 0.911, test acc 0.875
loss 0.215, train acc 0.918, test acc 0.892
Calculation Ability: 6034.0 examples/sec on cuda
